In [5]:
#!/bin/env python
# Author: Teja Koganti

# 02_calculate_tmb_from_mafdb.py
# This script does the following - 
#  1. Establishes a connection with sqlite 
#  2. Groupby column name "Tumor_Sample_Barcode" and counts the number of var
            # for each sample from table "mutect2"
#  3. Loads metadata file(pbta-histologies for PBTA)   
#  4. Use join to match up sample names and disease types and prints the final DF
#
# Note: requires pandas and sqlite to be installed, and expects python3
# conda install -c anaconda pandas
# conda install -c anaconda sqlite


import pandas as pd
import numpy as np
import sqlite3
import sys

# Will use argparse after first PR
metadatafile="/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/inputs/pbta-histologies.tsv"
#metadatafile="/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/inputs/PNOC008_histologies.tsv"
disease_col = "short_histology"
samplename_col = "Kids_First_Biospecimen_ID"
db_name="/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/output/var_db.sqlite"
#db_name = "/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/output/PNOC008WXS_var_db.sqlite"
target_bed_size=77462866    # This will be counted in bash script when it is fully setup 
out_tmb = "/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/output/pbta-snv-mutect2-tmbscores.target.txt"
#out_tmb = "/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/output/PNOC008_WXS_consensus-tmbscores.target.txt"
#intersectedbed_out_tmb = "/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/output/PNOC008_WXS_consensus-tmbscores.targetandcds.txt"
intersectedbed_out_tmb = "/Users/kogantit/Documents/git_repos/d3b-bix-analysis-toolkit/analyses/TMBanalysis/output/pbta-snv-mutect2-tmbscores.targetandcds.txt"
#db_tablename = "consensus_WXS"
#db_intersectbed_tablename = "consensus_WXS_intersectbed"
db_table_name = "mutect2"
db_intersectbed_table_name = "mutect2_intersectbed"
intersected_target_bed_size = 76472464



# Establiching a connection to the MAF db 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print("Error connecting to database")
    return(conn)    



##############################################################
##############Preparing dataframes for printing output########
# Creating and empty DF with columns for sample name and TMB
sample_tmb_colnames = ["Samplenames", "TMB"]
sample_tmb_df = pd.DataFrame(columns=sample_tmb_colnames)
sample_tmb_intersectbed_df = pd.DataFrame(columns=sample_tmb_colnames)
###############################################################



################################################################
#########Filling output dataframes#############################
################################################################
#
# Starting a database connection     
con = create_connection(db_name)
# Using GROUPBY to count vars in each sample from table mutect2 -> for targetbed
var_db = con.execute('select Tumor_Sample_Barcode,count(Tumor_Sample_Barcode) from '
                     +db_tablename+' GROUP BY  Tumor_Sample_Barcode')

sys.exit()

# Calculating TMB based on the results from MAF DB -> for targetbed
for each_sample in var_db.fetchall():
    samplename = each_sample[0]
    #print(samplename,  (int(each_sample[1])*1000000)/target_bed_size)
    sample_tmb_df = sample_tmb_df.append({"Samplenames" : samplename ,
                                          "TMB" : (int(each_sample[1])*1000000)/target_bed_size},ignore_index=True)
    
# Using GROUPBY to count vars in each sample from table mutect2 -> for intersectbed
var_db_intersectbed = con.execute('select Tumor_Sample_Barcode,count(Tumor_Sample_Barcode) from '
                                  +db_intersectbed_tablename+' GROUP BY  Tumor_Sample_Barcode')
# Calculating TMB based on the results from MAF DB -> for targetbed
for each_sample in var_db_intersectbed.fetchall():
    samplename = each_sample[0]
    #print(samplename,  (int(each_sample[1])*1000000)/target_bed_size)
    sample_tmb_intersectbed_df = sample_tmb_intersectbed_df.append({"Samplenames" : samplename ,
                                          "TMB" : (int(each_sample[1])*1000000)/intersected_target_bed_size} ,ignore_index=True)
    
################################################################    
    
    
    
#################################################################
########## Creating final TMB outputs ###########################
# Reading in metadata file to extract disease types
#metadata = pd.read_csv(metadatafile, sep="\t", index_col=False)
metadata = pd.read_csv(metadatafile, sep="\t", index_col=False)


#Checking of all samples within db_name are in the metadata file 
if not (np.in1d(sample_tmb_df["Samplenames"].astype(str), metadata[samplename_col].astype(str))).all():
    print("\n-----Error message: Samples not in metadata file!-----\n")
    sys.exit()
    
# Joining TMB results and metedata file and keeping only relevant columns 
final_tmb = sample_tmb_df.join(metadata.set_index(samplename_col), 
                               on="Samplenames")[["Samplenames",  "TMB", disease_col]]
final_tmb.to_csv(out_tmb, index=False)
intersectedbed_final_tmb = sample_tmb_intersectbed_df.join(metadata.set_index(samplename_col), 
                               on="Samplenames")[["Samplenames",  "TMB", disease_col]]
intersectedbed_final_tmb.to_csv(intersectedbed_out_tmb, index=False)
#################################################################


2.6.0


OperationalError: no such table: consensus_WXS